# Install Modules

!pip install selenium

!pip install python-dotenv

!pip install webdriver-manager

# LinkedIn Login Automation with Selenium

This notebook demonstrates how to automate logging into LinkedIn using Selenium WebDriver with Python. The script will:
- Log into LinkedIn using credentials stored in environment variables.
- Scroll the feed continuously.
- Click the "See new posts" button if it appears.

**Note:** Ensure you have set the environment variables `LINKEDIN_EMAIL` and `LINKEDIN_PASSWORD` with your LinkedIn credentials.

In [21]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
import os

## Function to Login to LinkedIn

The following function logs into LinkedIn and scrolls through the feed. It also clicks the "See new posts" button if it appears.

In [22]:
def login_to_linkedin_with_proxy(driver):
    try:
        # Set email here
        email = os.environ['LINKEDIN_EMAIL'] 
        email_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "session_key")))
        email_field.send_keys(email)
        
        # Set password here
        password = os.environ['LINKEDIN_PASSWORD']   
        password_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "session_password")))
        password_field.send_keys(password)

        login_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
        login_button.click()
        
        WebDriverWait(driver, 10).until(EC.url_contains("https://www.linkedin.com/feed/"))
        
        print("Login successful!")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            
            # Check if the button exists
            try:
                button = driver.find_element(By.CSS_SELECTOR, "button.artdeco-button.artdeco-button--secondary.mv5.t-14.t-black.t-normal")
                button.click()
                print("Clicked 'See new posts' button.")
            except NoSuchElementException:
                pass
        
    except Exception as e:
        print("Error during login:", e)
        
    finally:
        driver.quit()

## Running the Script

Before running the script, ensure that you have:
- Set up your environment variables `LINKEDIN_EMAIL` and `LINKEDIN_PASSWORD`.
- Installed the necessary packages using `pip install -r requirements.txt`.
- Downloaded the appropriate WebDriver for your browser and added it to your system PATH.

The following cell initializes the WebDriver and calls the `login_to_linkedin_with_proxy` function.

# LinkedIn Automation with Custom Chrome Driver

This notebook demonstrates how to use a custom Chrome WebDriver to automate tasks, such as rotating user agents, managing cookies, and performing logins using Selenium and environment variables. Ensure you have set the necessary environment variables before running the script.

In [23]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import random
import requests
import os
from dotenv import load_dotenv
load_dotenv()
#load env variable
LINKEDIN_LOGIN_URL= os.environ.get('LINKEDIN_LOGIN_URL')
LINKEDIN_EMAIL= os.environ.get('LINKEDIN_EMAIL')
LINKEDIN_PASSWORD= os.environ.get('LINKEDIN_PASSWORD')
PROXY_PORT= os.environ.get('PROXY_PORT')
PROXY_HOST= os.environ.get('PROXY_HOST')
PROXY_USERNAME= os.environ.get('PROXY_USERNAME')
PROXY_PASSWORD= os.environ.get('PROXY_PASSWORD')
CHROME_EXTENSION_URL= os.environ.get('CHROME_EXTENSION_URL')
FIREFOX_EXTENSION_URL= os.environ.get('FIREFOX_EXTENSION_URL')


ModuleNotFoundError: No module named 'webdriver_manager'

## Custom Chrome Driver Class

The following class defines a custom Chrome WebDriver that supports rotating User-Agent strings, managing cookies, using proxies, and downloading extensions.

In [12]:
class MyChromeDriver(Chrome):

  def __init__(self,user_agents=None, extension_url=None,is_enable_auto_flags=True, *args, **kwargs):
    
    proxy = kwargs.get('proxy', None) 
    # extension_url = kwargs.get('extension_url', None) 
    self.extension_name=None
    if extension_url is not None:
       self.extension_name=self.download_extension(extension_url)
    self.user_agents = user_agents or ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"]

    options = kwargs.get('options', None) or ChromeOptions()
    #set Proxy options
    if proxy is not None:
       
      options.add_argument('--proxy-server=%s:%s' % (proxy[0], proxy[1]))
    #user agents
    options.add_argument(f"user-agent={random.choice(self.user_agents)}")
    # Set custom options to disable automation flags
    if is_enable_auto_flags:
      options.add_experimental_option("useAutomationExtension", False)
      options.add_argument("--disable-infobars")
      options.add_argument("--disable-blink-features=AutomationControlled")
    if self.extension_name is not None:
       
      options.add_argument(f"--load-extension={os.path.abspath(os.path.join(os.getcwd(), self.extension_name))}")

    super().__init__(*args, **kwargs)

  def rotate_user_agent(self):
        """Rotates the User-Agent from the pre-defined list by setting a new one."""
        new_user_agent = random.choice(self.user_agents)
        self.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": new_user_agent})
        print(f"User-Agent rotated to: {new_user_agent}")

  def add_cookie(self, cookie_dict):
    """
        Adds a single cookie to your current session.
        :param cookie_dict: A dictionary object with details for the cookie: 
                            {'name': 'cookie_name', 'value': 'cookie_value', ...}
    """
    super().add_cookie(cookie_dict)

  def add_cookies(self, cookies):
    """
        Adds a list of cookies to your current session.
        :param cookies: A list of dictionaries where each dict contains 
                        details for a single cookie to be added.
    """
    for cookie in cookies:
      self.add_cookie(cookie)

  def get_cookie(self, name):

    """
        Get a single cookie by name. Returns the cookie if found, None if not.
        :param name: Name of the cookie.
    """
    return super().get_cookie(name)

  def get_all_cookies(self):
    """
        Returns a list of all cookies in the current session.
    """
    return super().get_cookies()

  def login_and_save_cookies(self, login_url, login_function, **login_args):
    """
        Perform login and then save the cookies post-login.
        :param login_url: The URL to navigate to for login.
        :param login_function: A function to call to perform login actions.
                               This function should accept a driver instance as its first argument,
                               followed by any *login_args. 
        :param login_args: Arguments necessary to perform the login provided to login_function.
    """
    # Navigate to the login page
    self.get(login_url)
    # Execute the login action, passing this driver instance and any other login args
    login_function(self, *login_args)
        # Save cookies after login
    return self.get_all_cookies()
  
  def download_extension(self,extension_url):
    response = requests.get(extension_url)
    name='extension.crx'
    with open(name, 'wb') as f:
        f.write(response.content)
    print("Extension downloaded successfully.")
    return name
    


__init__(self, user_agents=None, extension_url=None,,is_enable_auto_flags=True, *args, **kwargs): This initializes the Chrome driver instance. It accepts optional arguments for user agents and extension URL.

rotate_user_agent(self): This method rotates the User-Agent used by the browser, similar to the one in the Firefox driver. It can be used for web scraping or testing purposes.

add_cookie(self, cookie_dict): Adds a single cookie to the current session, similar to the Firefox driver method.

add_cookies(self, cookies): Adds a list of cookies to the current session, similar to the Firefox driver method.

get_cookie(self, name): Retrieves a single cookie by its name from the current session, similar to the Firefox driver method.

get_all_cookies(self): Retrieves all cookies present in the current session, similar to the Firefox driver method.

login_and_save_cookies(self, login_url, login_function, *login_args): This method automates the login process and then saves the cookies obtained after successful login. It's similar to the Firefox driver method.

download_extension(self, extension_url): Downloads a Chrome extension from a given URL. This can be used for automation tasks involving extensions, similar to the Firefox driver method.


## Below We are going to test driver with/without automation flag False/True

In [13]:
proxy_host = PROXY_HOST
proxy_port = PROXY_PORT

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0"
]

# Install the Chrome Driver
service = ChromeService(ChromeDriverManager().install())

# Download the extension CRX file
extension_url = CHROME_EXTENSION_URL
# download_extension(extension_url)

# # Set up Chrome options to load the extension
# chrome_options = Options()

# Initialize the driver with the loaded extension
driver = MyChromeDriver(service=service, user_agents=user_agents,is_enable_auto_flags=False,extension_url=extension_url)


driver


NameError: name 'PROXY_HOST' is not defined

## Let's try with automations Flags

In [14]:
# Initialize the driver with the loaded extension
driver = MyChromeDriver(service=service, user_agents=user_agents,extension_url=extension_url)


driver

NameError: name 'service' is not defined

## Let's Try to rotate User Agents

In [15]:
# Rotate user agents
driver.rotate_user_agent()  



NameError: name 'driver' is not defined

## Let's try to Login and lastly save Cookies

In [16]:
# Login URL
login_url = os.environ['LINKEDIN_LOGIN_URL'] 

# Perform login and save cookies
cookies = driver.login_and_save_cookies(login_url, login_to_linkedin_with_proxy)

# Add cookies to the driver
driver.add_cookies(cookies)

# Additional actions using the driver, if any
specific_cookie = driver.get_cookie('session_cookie_name')
all_cookies = driver.get_all_cookies()

all_cookies

KeyError: 'LINKEDIN_LOGIN_URL'

## Running the Script

Before running the script, ensure that you have:
- Installed the necessary packages using `pip install selenium requests`.
- Downloaded the appropriate WebDriver for Chrome and added it to your system PATH.

The following cell initializes the custom Chrome WebDriver and demonstrates how to use the methods defined in the class.


In [20]:
# Rotate user agents
driver.rotate_user_agent()

# Login URL
login_url = os.environ['LINKEDIN_LOGIN_URL']

# Perform login and save cookies
cookies = driver.login_and_save_cookies(login_url, login_to_linkedin_with_proxy)

# Add cookies to the driver
driver.add_cookies(cookies)

# Additional actions using the driver, if any
specific_cookie = driver.get_cookie('session_cookie_name')
all_cookies = driver.get_all_cookies()

# Print cookies for demonstration
print(specific_cookie)
print(all_cookies)

# Close the driver
driver.quit()

NameError: name 'driver' is not defined


# Here are some use cases where Selenium automation flags may be disabled:
## Data collection: 
To disable data collection, set the SE_AVOID_STATS environment variable to true or set avoid-stats = true in the configuration file
## Automation indicator: 
To avoid bot detection, disable the automation indicator WebDriver Flags
## Infobars: 
To disable the "Chrome is being controlled by automated test software" message, update the Selenium - Chrome profile with the disable-infobars argument